## Setup

In [1]:
import pickle
import spacy
import sys
from spacy import displacy
import benepar
import copy

In [2]:
# Set up and store NLP
nlp = spacy.load("en_core_web_md")
benepar.download('benepar_en3')
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})

with open('./nlp/nlp.pickle', 'wb') as f:
    pickle.dump(nlp, f)

[nltk_data] Downloading package benepar_en3 to
[nltk_data]     C:\Users\regan\AppData\Roaming\nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


In [3]:
# Load NLP
#sys.path.append('../')

with open('./nlp/nlp.pickle', 'rb') as f:
    nlp = pickle.load(f)

In [4]:
from app.classes.spec.symboleo_spec import Obligation
from app.lib.matchers.matchers_builder import MatchersBuilder
from app.lib.norm_proposition_updater import NormPropositionUpdater
from app.lib.sentence_obligation_converter import SentenceObligationConverter
from app.lib.sentence_summarizer import SentenceSummarizer
from app.templates.meat_sale.norms import meat_sale_norms
from app.templates.meat_sale.templates import meat_sale_templates

matchers_builder = MatchersBuilder(nlp)
matchers = matchers_builder.build()
norm_updater = NormPropositionUpdater()

converter = SentenceObligationConverter(nlp, matchers, norm_updater)
summarizer = SentenceSummarizer(nlp)

obligations: list[Obligation] = copy.deepcopy(meat_sale_norms['obligations'])
templates = meat_sale_templates['obligations']

In [8]:
USER_INPUT = False

DEFAULT_OBLIGATION_ID = 'delivery'
DEFAULT_COMPONENT_CHAR = 't'

COMPONENT_DICT = {
    'a': 'antecedent',
    't': 'trigger',
    'c': 'consequent'
}

SAMPLE_SENTENCES = [
    "It is a holiday",
    "The sky is blue",
    "It is not a holiday",
    "It is a full moon",
    "two plus two equals four",
    "the house is on fire",
    "Ted built a house",
    "it is expected to rain",
    "it is raining"
]

DEFAULT_SENTENCE = SAMPLE_SENTENCES[1]


## Gather user input

In [9]:
for ob in obligations:
    print(ob.id)
    print(f'- NL: {templates[ob.id]}')
    print(f'- SYM: {ob.to_sym()}')

delivery
- NL: The Seller shall deliver the Order in one delivery within <delDueDateDays> days to the Buyer at its warehouse
- SYM: delivery: O(seller, buyer, true, WhappensBefore(delivered, delivered.delDueDate))
payment
- NL: The Buyer shall pay <amt> (“amount”) in <curr> (“currency”) to the Seller before <payDueDate>.
- SYM: payment: O(buyer, seller, true, WhappensBefore(paid, paid.payDueDate))
latePayment
- NL: In the event of late payment of the amount owed due, the Buyer shall pay interests equal to <intRate>% of the amount owed
- SYM: latePayment: Happens(Violated(obligations.payment)) -> O(buyer, seller, true, Happens(paidLate))


In [10]:
# Choose the obligation
if USER_INPUT:
    # Choose the obligation
    selected_ob_id = input('Enter obligation id: ')
else:
    selected_ob_id = DEFAULT_OBLIGATION_ID

In [11]:
if USER_INPUT:
    selected_component_char = input('Select the component to update (t/a/c): ')
else:
    selected_component_char = DEFAULT_COMPONENT_CHAR

selected_component = COMPONENT_DICT[selected_component_char]

In [12]:
if USER_INPUT:
    sentence = input('Enter an extension sentence: ')
else:
    sentence = DEFAULT_SENTENCE

## Execution

In [13]:
# summarize the sentence
summarizer.summarize(sentence)


Sentence: The sky is blue

i     TEXT            POS             TAG             DEP             LEMMA           HEAD            ENT            
--------------------------------------------------------------------------------------------------------------
0     The             DET             DT              det             the             sky                            
1     sky             NOUN            NN              nsubj           sky             is                             
2     is              AUX             VBZ             ROOT            be              is                             
3     blue            ADJ             JJ              acomp           blue            is                             



             S              
      _______|_______        
     |               VP     
     |            ___|___    
     NP          |      ADJP
  ___|___        |       |   
 DT      NN     VBZ      JJ 
 |       |       |       |   
The     sky      is     blue





c:\Users\regan\anaconda3\lib\site-packages\torch\distributions\distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


In [14]:
obligation = [x for x in obligations if x.id == selected_ob_id][0] 

result = converter.convert(sentence, obligation, selected_component)

print(result.to_sym())

delivery: Happens(is_blue(sky)) -> O(seller, buyer, true, WhappensBefore(delivered, delivered.delDueDate))
